In [13]:
%load_ext sql
%sql sqlite:///CourseDataWarehouse.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


'Connected: @CourseDataWarehouse.db'

In [14]:
%%sql
ATTACH 'CourseData.db' AS CDDB

 * sqlite:///CourseDataWarehouse.db
(sqlite3.OperationalError) database CDDB is already in use
[SQL: ATTACH 'CourseData.db' AS CDDB]
(Background on this error at: http://sqlalche.me/e/e3q8)


In [15]:
%%sql
select * from CDDB.PROGRAMS LIMIT 1

 * sqlite:///CourseDataWarehouse.db
Done.


PID,program_code,program_name
1,AN,Asian Studies


In [63]:
%%sql 
DROP TABLE IF EXISTS PROGRAMS_DIM;
DROP TABLE IF EXISTS LOCATIONS_DIM;
DROP TABLE IF EXISTS TIMES_DIM;
DROP TABLE IF EXISTS FACULTY_DIM;
DROP TABLE IF EXISTS COURSE_CATALOGS_DIM;
DROP TABLE IF EXISTS COURSES_FACT;

CREATE TABLE PROGRAMS_DIM (
    PID INTEGER NOT NULL PRIMARY KEY,
    program_code TEXT NOT NULL,
    program_name TEXT NOT NULL
);


CREATE TABLE LOCATIONS_DIM (
    LID INTEGER PRIMARY KEY,
    location TEXT ,
    BuildingName TEXT

);


CREATE TABLE FACULTY_DIM (
    FID  INTEGER NOT NULL PRIMARY KEY,
    instructor_fname TEXT NOT NULL,
    instructor_lname TEXT NOT NULL
);


CREATE TABLE COURSE_CATALOGS_DIM (
    CID INTEGER PRIMARY KEY,
    Catalog_ID TEXT NOT NULL,
    Credits TEXT NOT NULL,
    Prereqs TEXT,
    Coreqs TEXT,
    Fees INTEGER,
    Description TEXT,
    Attributes TEXT
);



CREATE TABLE COURSES_FACT (
    CID INTEGER NULL ,
    CRN INTEGER NULL,
    SEMESTER TEXT NOT NULL ,
    YEAR INTEGER NOT NULL,
    Title TEXT NOT NULL,
    SECTION TEXT NOT NULL,
    Act INTEGER,
    Cap INTEGER,
    Rem INTEGER, 
    timecodes TEXT,
    PID INTEGER NOT NULL,
    LID INTEGER NOT NULL,
    FID INTEGER NOT NULL,
    catalog_ID INTEGER  NULL,
    FOREIGN KEY (PID) REFERENCES PROGRAMS_DIM (PID),
    FOREIGN KEY (LID) REFERENCES LOCATIONS_DIM (LID),
    FOREIGN KEY (FID) REFERENCES FACULTY_DIM (FID),
    FOREIGN KEY (CID) REFERENCES COURSE_CATALOGS_DIM (CID)
);

 * sqlite:///CourseDataWarehouse.db
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

In [64]:
%%sql
SELECT * FROM  COURSES_FACT

 * sqlite:///CourseDataWarehouse.db
Done.


CID,CRN,SEMESTER,YEAR,Title,SECTION,Act,Cap,Rem,timecodes,PID,LID,FID,catalog_ID


In [65]:
%%sql 


INSERT INTO PROGRAMS_DIM (program_name, program_code)
SELECT  program_name, program_code
FROM CDDB.PROGRAMS;


INSERT INTO LOCATIONS_DIM (location ,BuildingName )
SELECT  location , BuildingName
FROM CDDB.LOCATION;


INSERT INTO FACULTY_DIM (instructor_fname , instructor_lname)
SELECT  instructor_fname , instructor_lname
FROM CDDB.FACULTY;


INSERT INTO COURSE_CATALOGS_DIM (Catalog_ID, credits, Prereqs, Coreqs, Fees, Description, Attributes)
SELECT catalog_ID, credits, prereqs, coreqs, fees, description, attributes
FROM CDDB.COURSE;



INSERT INTO COURSES_FACT (CRN, SEMESTER , YEAR , TITLE, SECTION, ACT, CAP,REM, TIMECODES, PID, LID, FID, CID)
SELECT DISTINCT CDDB.CLASS_MEETINGS.CRN, CDDB.CLASS_MEETINGS.SEMESTER , CDDB.CLASS_MEETINGS.YEAR, CDDB.COURSE.course_title, CDDB.COURSE_OFFERING.SECTION, act, cap, rem, timecodes, PID, LOCATIONS_DIM.LID, FID,CDDB.COURSE.CID
FROM CDDB.COURSE_OFFERING
     JOIN CDDB.CLASS_MEETINGS USING  (COID)
    left JOIN LOCATIONS_DIM USING (LID)
    left JOIN CDDB.COURSE USING (CID)
    left JOIN PROGRAMS_DIM USING (PID)
    left JOIN FACULTY_DIM USING (FID)
    JOIN COURSE_CATALOGS_DIM USING (CID);
    
    

    
   

 * sqlite:///CourseDataWarehouse.db
83 rows affected.
207 rows affected.
1104 rows affected.
4440 rows affected.
(sqlite3.OperationalError) ambiguous column name: CRN
[SQL: INSERT INTO COURSES_FACT (CRN, SEMESTER , YEAR , TITLE, SECTION, ACT, CAP,REM, TIMECODES, PID, LID, FID, CID)
SELECT DISTINCT CDDB.CLASS_MEETINGS.CRN, CDDB.CLASS_MEETINGS.SEMESTER , CDDB.CLASS_MEETINGS.YEAR, CDDB.COURSE.course_title, CDDB.COURSE_OFFERING.SECTION, act, cap, rem, timecodes, PID, LOCATIONS_DIM.LID, FID,CDDB.COURSE.CID, COUNT(CRN)
FROM CDDB.COURSE_OFFERING
    inner JOIN CDDB.CLASS_MEETINGS USING  (COID)
    inner JOIN LOCATIONS_DIM USING (LID)
    inner JOIN CDDB.COURSE USING (CID)
    inner JOIN PROGRAMS_DIM USING (PID)
    inner JOIN FACULTY_DIM USING (FID)
    inner JOIN COURSE_CATALOGS_DIM USING (CID);]
(Background on this error at: http://sqlalche.me/e/e3q8)


In [67]:
INSERT INTO COURSES_FACT (CRN, SEMESTER , YEAR , TITLE, SECTION, ACT, CAP,REM, TIMECODES, PID, LID, FID, CID)
SELECT DISTINCT CDDB.CLASS_MEETINGS.CRN, CDDB.CLASS_MEETINGS.SEMESTER , CDDB.CLASS_MEETINGS.YEAR, CDDB.COURSE.course_title, CDDB.COURSE_OFFERING.SECTION, act, cap, rem, timecodes, PID, LOCATIONS_DIM.LID, FID,CDDB.COURSE.CID
FROM CDDB.COURSE_OFFERING
     JOIN CDDB.CLASS_MEETINGS USING  (COID)
    left JOIN LOCATIONS_DIM USING (LID)
    left JOIN CDDB.COURSE USING (CID)
    left JOIN PROGRAMS_DIM USING (PID)
    left JOIN FACULTY_DIM USING (FID)
    JOIN COURSE_CATALOGS_DIM USING (CID);

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 5)

In [51]:
%%sql
SELECT FID, instructor_lname,instructor_fname,COURSE_CATALOGS_DIM.Catalog_ID,program_name, Title ,sum(act), sum(cap), sum(rem) , YEAR,SEMESTER , COUNT(CRN) as COURSESTaught
FROM FACULTY_DIM
    JOIN COURSES_FACT USING (FID)
    JOIN COURSE_CATALOGS_DIM USING (CID)
    JOIN PROGRAMS_DIM USING (PID)
WHERE (instructor_lname like '%Huntley%')
GROUP BY title
ORDER BY COURSESTaught DESC


 * sqlite:///CourseDataWarehouse.db
Done.


FID,instructor_lname,instructor_fname,Catalog_ID,program_name,Title,sum(act),sum(cap),sum(rem),YEAR,SEMESTER,COURSESTaught
275,Huntley,Christopher L.,OM 0101,Operations Management,Operations Management,102,100,-2,2019,Spring,4
275,Huntley,Christopher L.,IS 0240,Information Systems,Systems Analysis and Logical Design,45,50,5,2017,Fall,2
275,Huntley,Christopher L.,IS 0260,Information Systems,Database Systems,24,25,1,2018,Spring,1
275,Huntley,Christopher L.,IS 0510,Information Systems,Databases for Business Analytics,18,20,2,2017,Fall,1
275,Huntley,Christopher L.,IS 0505,Information Systems,Python for Business Analytics,28,20,-8,2017,Fall,1
275,Huntley,Christopher L.,IS 0320,Information Systems,Systems Design and Implementation,20,25,5,2018,Spring,1


In [52]:
%%sql
SELECT DISTINCT COUNT(CRN), COUNT(CID), COUNT(CRN)
FROM COURSES_FACT;

 * sqlite:///CourseDataWarehouse.db
Done.


COUNT(CRN),COUNT(CID),COUNT(CRN)_1
5207,5207,5207


In [53]:
%%sql

SELECT CRN,COURSE_CATALOGS_DIM.Catalog_ID, program_name,  instructor_lname,instructor_fname, YEAR,location ,BuildingName
FROM COURSES_FACT
    JOIN PROGRAMS_DIM USING (PID)
    JOIN COURSE_CATALOGS_DIM USING (CID)
    JOIN FACULTY_DIM USING (FID)
    JOIN LOCATIONS_DIM USING (LID)
WHERE program_name like 'iNFORMATION%'  or program_name like 'Operations Management'  and YEAR = 2019
GROUP BY CRN
ORDER BY program_name DESC

    

 * sqlite:///CourseDataWarehouse.db
Done.


CRN,Catalog_ID,program_name,instructor_lname,instructor_fname,YEAR,location,BuildingName
37200,OM 0101,Operations Management,Huntley,Christopher L.,2019,DSB 110B,Dolan School of Business
37201,OM 0101,Operations Management,Huntley,Christopher L.,2019,DSB 110B,Dolan School of Business
37202,OM 0101,Operations Management,Lewis,Neal A.,2019,DSB 110B,Dolan School of Business
37203,OM 0101,Operations Management,Remediani,Christopher J.,2019,DSB 107,Dolan School of Business
37222,OM 0101,Operations Management,Lewis,Neal A.,2019,DSB 112,Dolan School of Business
37269,OM 0101,Operations Management,Remediani,Christopher J.,2019,DSB 112,Dolan School of Business
37579,OM 0101,Operations Management,He,Xin Xin,2019,DSB 110A,Dolan School of Business
37664,OM 0101,Operations Management,He,Xin Xin,2019,DSB 110A,Dolan School of Business
38082,OM 0101,Operations Management,He,Xin Xin,2019,DSB 110A,Dolan School of Business
11834,IS 0500,Information Systems,Ozcelik,Yasin,2018,DSB 106,Dolan School of Business


In [71]:
%%sql
SELECT COUNT(DISTINCT instructor_lname) 
FROM COURSES_FACT
    JOIN PROGRAMS_DIM USING (PID)
    JOIN COURSE_CATALOGS_DIM USING (CID)
    JOIN FACULTY_DIM USING (FID)
    JOIN LOCATIONS_DIM USING (LID)
WHERE YEAR = 2019

 * sqlite:///CourseDataWarehouse.db
Done.


COUNT(DISTINCT instructor_lname)
0


In [12]:
%%sql
vacuum; 

 * sqlite:///CourseDataWarehouse.db
Done.


[]